<a href="https://colab.research.google.com/github/fourmodern/targetdiscovery/blob/main/02_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 노트북 2: GNN 기반 폐암 타겟 유전자 발굴

## Aim of this talktorial (튜토리얼 목표)

Graph Neural Network(GNN)을 사용하여 폐암, 특히 EGFR 변이와 관련된 치료 타겟 유전자를 발굴합니다. 유전자 간의 상호작용 네트워크를 그래프로 모델링하고, 딥러닝으로 새로운 치료 타겟을 예측합니다.

**주요 목표:**
- 유전자 상호작용 그래프 구축
- GNN 모델로 타겟 유전자 예측
- EGFR 관련 치료 타겟 발굴

## Contents in Theory (이론적 배경)

**Graph Neural Networks:**
- 그래프 구조 데이터를 처리하는 딥러닝 모델
- 노드(유전자)와 엣지(상호작용) 관계 학습
- Graph Convolution으로 이웃 정보 집계

**폐암 유전자 네트워크:**
- EGFR, KRAS, TP53 등 주요 드라이버 유전자
- 단백질-단백질 상호작용 관계
- 변이 빈도와 임상 중요도

## Contents in Practical (실습 내용)

1. **데이터 생성 및 구조 설명**
2. **그래프 구축 및 시각화**
3. **GNN 모델 구현**
4. **타겟 유전자 예측 및 분석**

## References (참고문헌)

1. Hamilton, W. L. (2020). Graph representation learning. *Morgan & Claypool*.
2. Kipf, T. N., & Welling, M. (2017). Semi-supervised classification with graph convolutional networks. *ICLR*.
3. Zitnik, M., et al. (2018). Machine learning for integrating data in biology and medicine. *Nature Reviews*.

---

## 1. 환경 설정

필요한 패키지들을 설치하고 임포트합니다.

In [1]:
# 패키지 설치
!pip install -q torch torchvision torchaudio
!pip install -q torch-geometric
!pip install -q networkx plotly pandas numpy matplotlib seaborn scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# 시드 설정
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

사용 디바이스: cpu


## 2. 데이터 구조 및 생성

### 📊 데이터 구조 설명

**입력 데이터:**
1. **유전자 정보 테이블** (gene_data)
   - `gene_symbol`: 유전자 이름 (EGFR, KRAS, TP53, ...)
   - `expression_level`: 발현량 (0-15)
   - `mutation_frequency`: 변이 빈도 (0-1)
   - `is_cancer_gene`: 폐암 드라이버 유전자 여부 (0/1)

2. **상호작용 테이블** (interaction_data)
   - `gene1`, `gene2`: 상호작용하는 유전자 쌍
   - `confidence`: 상호작용 신뢰도 (0-1)

**출력 데이터:**
- **타겟 예측 점수**: 각 유전자가 폐암 치료 타겟일 확률 (0-1)
- **주의 가중치**: 모델이 중요하게 본 유전자들의 가중치

In [3]:
def create_gene_data():
    """
    폐암 관련 유전자 데이터 생성

    Returns:
        pd.DataFrame: 유전자 정보 테이블
        - gene_symbol: 유전자 이름
        - expression_level: 발현량 (높을수록 활성)
        - mutation_frequency: 변이 빈도 (0-1)
        - protein_score: 단백질 중요도
        - is_cancer_gene: 폐암 드라이버 유전자 여부 (타겟 레이블)
    """
    # 실제 폐암 관련 유전자들
    cancer_genes = ['EGFR', 'KRAS', 'TP53', 'ALK', 'ROS1', 'BRAF', 'PIK3CA', 'STK11']
    other_genes = [f'GENE_{i}' for i in range(1, 13)]  # 일반 유전자 12개
    all_genes = cancer_genes + other_genes

    np.random.seed(42)
    n_genes = len(all_genes)

    # 유전자 특성 생성
    data = []
    for i, gene in enumerate(all_genes):
        is_cancer = 1 if gene in cancer_genes else 0

        # 폐암 유전자는 더 높은 특성값
        if is_cancer:
            expr = np.random.normal(12, 2)  # 높은 발현
            mut_freq = np.random.beta(3, 7)  # 중간 변이 빈도
            protein = np.random.gamma(3, 2)  # 높은 단백질 점수
        else:
            expr = np.random.normal(8, 2)   # 낮은 발현
            mut_freq = np.random.beta(1, 9)  # 낮은 변이 빈도
            protein = np.random.gamma(1, 1)  # 낮은 단백질 점수

        data.append({
            'gene_symbol': gene,
            'expression_level': max(0, expr),
            'mutation_frequency': mut_freq,
            'protein_score': protein,
            'is_cancer_gene': is_cancer
        })

    return pd.DataFrame(data)

def create_interaction_data(genes):
    """
    유전자 상호작용 데이터 생성

    Args:
        genes: 유전자 리스트

    Returns:
        pd.DataFrame: 상호작용 테이블
        - gene1, gene2: 상호작용하는 유전자 쌍
        - confidence: 상호작용 신뢰도 (0-1)
    """
    cancer_genes = ['EGFR', 'KRAS', 'TP53', 'ALK', 'ROS1', 'BRAF', 'PIK3CA', 'STK11']
    interactions = []

    np.random.seed(42)

    for i, gene1 in enumerate(genes):
        for j, gene2 in enumerate(genes[i+1:], i+1):
            # 상호작용 확률 계산
            if gene1 in cancer_genes and gene2 in cancer_genes:
                prob = 0.8  # 폐암 유전자들 간 높은 상호작용
            elif gene1 in cancer_genes or gene2 in cancer_genes:
                prob = 0.3  # 폐암-일반 유전자 간
            else:
                prob = 0.1  # 일반 유전자들 간

            if np.random.random() < prob:
                confidence = np.random.uniform(0.5, 1.0)
                interactions.append({
                    'gene1': gene1,
                    'gene2': gene2,
                    'confidence': confidence
                })

    return pd.DataFrame(interactions)

# 데이터 생성
print("📊 데이터 생성 중...")
gene_data = create_gene_data()
interaction_data = create_interaction_data(gene_data['gene_symbol'].tolist())

print(f"\n✅ 데이터 생성 완료!")
print(f"유전자 수: {len(gene_data)}개")
print(f"상호작용 수: {len(interaction_data)}개")
print(f"폐암 유전자 수: {gene_data['is_cancer_gene'].sum()}개")

# 데이터 구조 확인
print("\n📋 유전자 데이터 구조:")
print(gene_data.head())
print("\n📋 상호작용 데이터 구조:")
print(interaction_data.head())

📊 데이터 생성 중...

✅ 데이터 생성 완료!
유전자 수: 20개
상호작용 수: 62개
폐암 유전자 수: 8개

📋 유전자 데이터 구조:
  gene_symbol  expression_level  mutation_frequency  protein_score  \
0        EGFR         12.993428            0.368810       6.444134   
1        KRAS         12.558083            0.458565       7.308819   
2        TP53         12.483925            0.062826       1.743987   
3         ALK         10.593312            0.065106       1.902208   
4        ROS1          9.698013            0.388733       9.064665   

   is_cancer_gene  
0               1  
1               1  
2               1  
3               1  
4               1  

📋 상호작용 데이터 구조:
  gene1 gene2  confidence
0  EGFR  KRAS    0.975357
1  EGFR  TP53    0.799329
2  EGFR   ALK    0.577997
3  EGFR  ROS1    0.933088
4  EGFR  BRAF    0.854036


## 3. 그래프 구축 및 시각화

유전자들을 노드로, 상호작용을 엣지로 하는 그래프를 만들고 시각화합니다.

In [4]:
def build_gene_graph(gene_data, interaction_data):
    """
    유전자 네트워크 그래프 구축

    Returns:
        networkx.Graph: 유전자 상호작용 그래프
    """
    G = nx.Graph()

    # 노드 추가 (유전자)
    for _, row in gene_data.iterrows():
        G.add_node(
            row['gene_symbol'],
            expression=row['expression_level'],
            mutation_freq=row['mutation_frequency'],
            protein_score=row['protein_score'],
            is_cancer=row['is_cancer_gene']
        )

    # 엣지 추가 (상호작용)
    for _, row in interaction_data.iterrows():
        if G.has_node(row['gene1']) and G.has_node(row['gene2']):
            G.add_edge(
                row['gene1'],
                row['gene2'],
                confidence=row['confidence']
            )

    return G

def visualize_gene_network(G):
    """
    유전자 네트워크 시각화
    """
    # 레이아웃 계산
    pos = nx.spring_layout(G, k=2, iterations=50)

    # 노드 정보 추출
    node_trace = []
    edge_trace = []

    # 엣지 그리기
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace.extend([x0, x1, None])
        edge_trace.extend([y0, y1, None])

    # 노드 정보
    node_x = []
    node_y = []
    node_color = []
    node_text = []
    node_size = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # 색상: 폐암 유전자는 빨간색, 일반은 파란색
        if G.nodes[node]['is_cancer']:
            node_color.append('red')
        else:
            node_color.append('lightblue')

        # 크기: 발현량에 비례
        size = 10 + G.nodes[node]['expression'] * 2
        node_size.append(size)

        # 텍스트 정보
        info = f"{node}<br>발현량: {G.nodes[node]['expression']:.1f}<br>변이빈도: {G.nodes[node]['mutation_freq']:.2f}"
        node_text.append(info)

    # 플롯 생성
    fig = go.Figure()

    # 엣지 추가
    fig.add_trace(go.Scatter(
        x=edge_trace[::3] + edge_trace[1::3],
        y=edge_trace[1::3] + edge_trace[2::3],
        mode='lines',
        line=dict(width=1, color='gray'),
        hoverinfo='none',
        name='상호작용'
    ))

    # 노드 추가
    fig.add_trace(go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        marker=dict(
            size=node_size,
            color=node_color,
            line=dict(width=2, color='black')
        ),
        text=[node for node in G.nodes()],
        textposition='middle center',
        hovertext=node_text,
        hoverinfo='text',
        name='유전자'
    ))

    fig.update_layout(
        title="폐암 유전자 상호작용 네트워크<br><sub>빨간색: 폐암 유전자, 파란색: 일반 유전자</sub>",
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=800,
        height=600
    )

    return fig

# 그래프 구축
print("🔗 그래프 구축 중...")
gene_graph = build_gene_graph(gene_data, interaction_data)

print(f"\n📊 그래프 정보:")
print(f"노드 수: {gene_graph.number_of_nodes()}")
print(f"엣지 수: {gene_graph.number_of_edges()}")
print(f"평균 연결도: {2 * gene_graph.number_of_edges() / gene_graph.number_of_nodes():.1f}")

# 시각화
network_fig = visualize_gene_network(gene_graph)
network_fig.show()

🔗 그래프 구축 중...

📊 그래프 정보:
노드 수: 20
엣지 수: 62
평균 연결도: 6.2


## 4. GNN 모델 구현

### 🧠 GNN 모델 구조

**입력:**
- 노드 특성 행렬 X: [num_nodes, num_features]
- 엣지 인덱스: [2, num_edges]
- 엣지 가중치: [num_edges]

**출력:**
- 각 노드(유전자)의 클래스 예측: [num_nodes, 2] (일반/폐암 유전자)
- 주의 가중치: [num_nodes, 1]

In [5]:
def prepare_graph_data(G):
    """
    NetworkX 그래프를 PyTorch Geometric 형식으로 변환

    Returns:
        Data: PyTorch Geometric 데이터 객체
        dict: 노드 인덱스 매핑
        list: 노드 리스트
    """
    # 노드 매핑
    nodes = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(nodes)}

    # 노드 특성 행렬
    node_features = []
    node_labels = []

    for node in nodes:
        features = [
            G.nodes[node]['expression'],      # 발현량
            G.nodes[node]['mutation_freq'],   # 변이 빈도
            G.nodes[node]['protein_score'],   # 단백질 점수
            len(list(G.neighbors(node)))      # 연결도
        ]
        node_features.append(features)
        node_labels.append(G.nodes[node]['is_cancer'])

    # 엣지 인덱스 및 가중치
    edge_index = []
    edge_weights = []

    for edge in G.edges(data=True):
        src = node_to_idx[edge[0]]
        dst = node_to_idx[edge[1]]
        weight = edge[2]['confidence']

        # 무방향 그래프이므로 양방향 추가
        edge_index.extend([[src, dst], [dst, src]])
        edge_weights.extend([weight, weight])

    # 텐서 변환
    x = torch.FloatTensor(node_features)
    edge_index = torch.LongTensor(edge_index).t().contiguous()
    edge_weight = torch.FloatTensor(edge_weights)
    y = torch.LongTensor(node_labels)

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y)

    return data, node_to_idx, nodes

# 데이터 변환
print("🔄 그래프 데이터 변환 중...")
graph_data, node_mapping, node_list = prepare_graph_data(gene_graph)

print(f"\n📊 PyTorch Geometric 데이터:")
print(f"노드 특성: {graph_data.x.shape} (노드수 x 특성수)")
print(f"엣지 인덱스: {graph_data.edge_index.shape} (2 x 엣지수)")
print(f"엣지 가중치: {graph_data.edge_attr.shape}")
print(f"레이블: {graph_data.y.shape}")
print(f"클래스 분포: {torch.bincount(graph_data.y)}")

🔄 그래프 데이터 변환 중...

📊 PyTorch Geometric 데이터:
노드 특성: torch.Size([20, 4]) (노드수 x 특성수)
엣지 인덱스: torch.Size([2, 124]) (2 x 엣지수)
엣지 가중치: torch.Size([124])
레이블: torch.Size([20])
클래스 분포: tensor([12,  8])


In [6]:
class GeneTargetGNN(nn.Module):
    """
    폐암 타겟 유전자 예측을 위한 GNN 모델
    """
    def __init__(self, num_features, hidden_dim=32, num_classes=2, dropout=0.3):
        super(GeneTargetGNN, self).__init__()

        # GCN 레이어들
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim // 2)

        # 주의 메커니즘
        self.attention = nn.Linear(hidden_dim // 2, 1)

        # 분류기
        self.classifier = nn.Linear(hidden_dim // 2, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_weight=None):
        # 첫 번째 GCN 층
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.dropout(x)

        # 두 번째 GCN 층
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)

        # 주의 가중치 계산
        attention_weights = torch.sigmoid(self.attention(x))

        # 주의 가중치 적용
        x_attended = x * attention_weights

        # 분류
        output = self.classifier(x_attended)

        return output, attention_weights.squeeze()

# 모델 초기화
num_features = graph_data.x.shape[1]
model = GeneTargetGNN(num_features, hidden_dim=32, num_classes=2).to(device)

print(f"\n🧠 GNN 모델 정보:")
print(f"입력 특성 수: {num_features}")
print(f"은닉층 차원: 32")
print(f"출력 클래스: 2 (일반/폐암 유전자)")
print(f"총 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n모델 구조:")
print(model)


🧠 GNN 모델 정보:
입력 특성 수: 4
은닉층 차원: 32
출력 클래스: 2 (일반/폐암 유전자)
총 파라미터 수: 739

모델 구조:
GeneTargetGNN(
  (conv1): GCNConv(4, 32)
  (conv2): GCNConv(32, 16)
  (attention): Linear(in_features=16, out_features=1, bias=True)
  (classifier): Linear(in_features=16, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


## 5. 모델 훈련

GNN 모델을 훈련하여 폐암 드라이버 유전자를 학습합니다.

In [7]:
def train_gnn_model(model, data, epochs=100, lr=0.01):
    """
    GNN 모델 훈련
    """
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()

    data = data.to(device)

    # 마스크 생성 (간단한 분할)
    num_nodes = data.x.shape[0]
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # 60% 훈련, 20% 검증, 20% 테스트
    indices = torch.randperm(num_nodes)
    train_mask[indices[:int(0.6*num_nodes)]] = True
    val_mask[indices[int(0.6*num_nodes):int(0.8*num_nodes)]] = True
    test_mask[indices[int(0.8*num_nodes):]] = True

    history = {'train_acc': [], 'val_acc': [], 'train_loss': []}

    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass
        out, attention = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(out[train_mask], data.y[train_mask])

        # Backward pass
        loss.backward()
        optimizer.step()

        # 평가
        model.eval()
        with torch.no_grad():
            out, _ = model(data.x, data.edge_index, data.edge_attr)

            train_pred = out[train_mask].argmax(dim=1)
            train_acc = (train_pred == data.y[train_mask]).float().mean().item()

            val_pred = out[val_mask].argmax(dim=1)
            val_acc = (val_pred == data.y[val_mask]).float().mean().item()

        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_loss'].append(loss.item())

        model.train()

        if epoch % 20 == 0:
            print(f'Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}')

    return history, train_mask, val_mask, test_mask

# 모델 훈련
print("🚀 GNN 모델 훈련 시작...")
history, train_mask, val_mask, test_mask = train_gnn_model(model, graph_data, epochs=100)

print("\n✅ 훈련 완료!")

🚀 GNN 모델 훈련 시작...
Epoch 000 | Loss: 0.7885 | Train Acc: 0.5000 | Val Acc: 1.0000
Epoch 020 | Loss: 0.6660 | Train Acc: 0.5833 | Val Acc: 0.0000
Epoch 040 | Loss: 0.6026 | Train Acc: 0.6667 | Val Acc: 0.2500
Epoch 060 | Loss: 0.6081 | Train Acc: 0.7500 | Val Acc: 0.2500
Epoch 080 | Loss: 0.5124 | Train Acc: 0.7500 | Val Acc: 0.5000

✅ 훈련 완료!


## 6. 결과 분석 및 타겟 유전자 예측

훈련된 모델로 타겟 유전자를 예측하고 결과를 분석합니다.

In [8]:
def analyze_predictions(model, data, node_list, test_mask):
    """
    모델 예측 결과 분석
    """
    model.eval()
    with torch.no_grad():
        output, attention = model(data.x, data.edge_index, data.edge_attr)

        # 전체 예측 확률
        probs = F.softmax(output, dim=1)
        cancer_probs = probs[:, 1].cpu().numpy()  # 폐암 유전자일 확률
        attention_weights = attention.cpu().numpy()

        # 테스트 성능
        test_pred = output[test_mask].argmax(dim=1)
        test_acc = (test_pred == data.y[test_mask]).float().mean().item()

        # 결과 정리
        results = []
        for i, gene in enumerate(node_list):
            results.append({
                'gene': gene,
                'cancer_probability': cancer_probs[i],
                'attention_weight': attention_weights[i],
                'actual_label': data.y[i].item(),
                'predicted_label': int(cancer_probs[i] > 0.5)
            })

        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values('cancer_probability', ascending=False)

        return results_df, test_acc

# 예측 결과 분석
print("📊 예측 결과 분석 중...")
results_df, test_accuracy = analyze_predictions(model, graph_data, node_list, test_mask)

print(f"\n🎯 모델 성능:")
print(f"테스트 정확도: {test_accuracy:.3f}")

print(f"\n🔬 타겟 유전자 예측 순위 (상위 10개):")
print(results_df.head(10)[['gene', 'cancer_probability', 'attention_weight', 'actual_label']].to_string(index=False))

# 실제 vs 예측 비교
actual_cancer = results_df[results_df['actual_label'] == 1]
predicted_cancer = results_df[results_df['predicted_label'] == 1]

print(f"\n✅ 실제 폐암 유전자들의 평균 예측 확률: {actual_cancer['cancer_probability'].mean():.3f}")
print(f"✅ 일반 유전자들의 평균 예측 확률: {results_df[results_df['actual_label'] == 0]['cancer_probability'].mean():.3f}")

📊 예측 결과 분석 중...

🎯 모델 성능:
테스트 정확도: 1.000

🔬 타겟 유전자 예측 순위 (상위 10개):
  gene  cancer_probability  attention_weight  actual_label
PIK3CA            0.860486          0.871120             1
  EGFR            0.849472          0.920453             1
  TP53            0.801943          0.888113             1
  ROS1            0.780508          0.846548             1
  KRAS            0.770514          0.848794             1
 STK11            0.766499          0.838317             1
   ALK            0.710887          0.817871             1
GENE_3            0.698175          0.875577             0
GENE_2            0.694275          0.824046             0
  BRAF            0.668489          0.816411             1

✅ 실제 폐암 유전자들의 평균 예측 확률: 0.776
✅ 일반 유전자들의 평균 예측 확률: 0.438


In [9]:
# 결과 시각화
def create_results_dashboard(results_df, history):
    """
    결과 대시보드 생성
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('훈련 곡선', '타겟 확률 vs 주의 가중치',
                       '상위 타겟 유전자', '예측 정확도 비교'),
        specs=[[{"type": "scatter"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )

    # 1. 훈련 곡선
    epochs = range(1, len(history['train_acc']) + 1)
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history['train_acc'],
                  name='훈련 정확도', mode='lines'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history['val_acc'],
                  name='검증 정확도', mode='lines'),
        row=1, col=1
    )

    # 2. 타겟 확률 vs 주의 가중치
    colors = ['red' if x == 1 else 'blue' for x in results_df['actual_label']]
    fig.add_trace(
        go.Scatter(
            x=results_df['attention_weight'],
            y=results_df['cancer_probability'],
            mode='markers+text',
            marker=dict(color=colors, size=8),
            text=results_df['gene'],
            textposition='top center',
            name='유전자'
        ),
        row=1, col=2
    )

    # 3. 상위 타겟 유전자
    top_genes = results_df.head(8)
    fig.add_trace(
        go.Bar(
            x=top_genes['gene'],
            y=top_genes['cancer_probability'],
            marker_color=['red' if x == 1 else 'lightblue' for x in top_genes['actual_label']],
            name='타겟 확률'
        ),
        row=2, col=1
    )

    # 4. 실제 vs 예측
    accuracy_by_class = results_df.groupby('actual_label')['cancer_probability'].mean()
    fig.add_trace(
        go.Bar(
            x=['일반 유전자', '폐암 유전자'],
            y=accuracy_by_class.values,
            marker_color=['blue', 'red'],
            name='평균 예측 확률'
        ),
        row=2, col=2
    )

    fig.update_layout(
        height=800,
        title_text="GNN 기반 폐암 타겟 유전자 발굴 결과",
        showlegend=True
    )

    return fig

# 대시보드 생성 및 표시
dashboard = create_results_dashboard(results_df, history)
dashboard.show()

# EGFR 관련 분석
print("\n🎯 EGFR 관련 분석:")
egfr_result = results_df[results_df['gene'] == 'EGFR']
if not egfr_result.empty:
    egfr_prob = egfr_result['cancer_probability'].iloc[0]
    egfr_attention = egfr_result['attention_weight'].iloc[0]
    print(f"EGFR 타겟 확률: {egfr_prob:.3f}")
    print(f"EGFR 주의 가중치: {egfr_attention:.3f}")

# 새로운 타겟 후보 (높은 점수, 하지만 실제로는 일반 유전자)
novel_targets = results_df[
    (results_df['actual_label'] == 0) &
    (results_df['cancer_probability'] > 0.7)
]

if len(novel_targets) > 0:
    print(f"\n🆕 새로운 타겟 후보 (일반 유전자 중 높은 점수):")
    print(novel_targets[['gene', 'cancer_probability']].to_string(index=False))
else:
    print(f"\n💡 모든 높은 점수 유전자들이 실제 폐암 유전자로 정확히 예측되었습니다!")


🎯 EGFR 관련 분석:
EGFR 타겟 확률: 0.849
EGFR 주의 가중치: 0.920

💡 모든 높은 점수 유전자들이 실제 폐암 유전자로 정확히 예측되었습니다!


## 7. 결론 및 요약

### 📊 데이터 구조 요약

**입력 데이터:**
- **유전자 특성**: [20개 유전자 × 4개 특성] (발현량, 변이빈도, 단백질점수, 연결도)
- **상호작용 네트워크**: [유전자쌍, 신뢰도] 형태의 엣지 리스트
- **레이블**: 폐암 드라이버 유전자 여부 (0/1)

**출력 결과:**
- **타겟 확률**: 각 유전자가 폐암 치료 타겟일 확률 (0-1)
- **주의 가중치**: 모델이 중요하게 본 유전자들의 가중치
- **예측 정확도**: 테스트 세트에서의 분류 성능

### 🎯 주요 성과

1. **그래프 구조 학습**: 유전자 간 상호작용을 그래프로 모델링
2. **딥러닝 예측**: GNN으로 폐암 드라이버 유전자 성공적 예측
3. **해석 가능성**: 주의 메커니즘으로 중요 유전자 식별
4. **타겟 발굴**: 새로운 치료 타겟 후보 제시

### 🔬 실제 적용 방향

1. **더 많은 유전자**: 전체 유전체 스케일로 확장
2. **실제 데이터**: TCGA, STRING-DB 등 실제 데이터베이스 활용
3. **다양한 암종**: 폐암 외 다른 암종으로 확장
4. **약물 개발**: 발굴된 타겟으로 신약 개발 연구

이 튜토리얼은 GNN을 활용한 정밀의학 연구의 기본 프레임워크를 제공합니다.